# Assigning Sessions to Project

In [2]:
import datajoint as dj
from datetime import datetime
dj.conn()

Connecting root@localhost:3306


DataJoint connection (connected) root@localhost:3306

In [3]:
from ephys import acquisition, animal

In [4]:
acquisition.Session()

animal_id,datasource_id,session_time start time of this session,session_note
00342c6d86e97d43,0,2019-07-04 20:35:12,=BLOB=
40bfa03a7a7018cc,0,2018-07-03 20:32:28,=BLOB=
575882867c176190,0,2019-09-04 10:39:09,=BLOB=
575882867c176190,0,2019-09-24 09:16:14,=BLOB=
575882867c176190,0,2019-09-24 09:33:50,=BLOB=
61fd2ac184c13c73,0,2017-10-02 15:54:25,=BLOB=
6a61369265b193e8,0,2019-03-27 10:36:57,=BLOB=


In [5]:
acquisition.Project()

project_name,project_desc,publication e.g. publication doi
proj1,,
proj2,,


In [6]:
acquisition.ProjectSession()

project_name,animal_id,datasource_id,session_time start time of this session


### First select the session(s) to be assigned to a particular project

In [7]:
acquisition.Session & (acquisition.Session.Experimenter & 'experimenter = "torgeirw"')

animal_id,datasource_id,session_time start time of this session,session_note
724240a0a756020f,0,2018-04-24 10:25:23,=BLOB=


In [8]:
session_keys = (acquisition.Session & (acquisition.Session.Experimenter & 'experimenter = "torgeirw"')).fetch('KEY')

### Then insert those sessions into the ***ProjectSession*** table, providing a project name

In [9]:
project_name = 'proj1'

In [10]:
acquisition.ProjectSession.insert(dict(session, project_name=project_name) for session in session_keys)

In [11]:
acquisition.ProjectSession()

project_name,animal_id,datasource_id,session_time start time of this session
proj1,724240a0a756020f,0,2018-04-24 10:25:23


## Or doing that all on the server side, no fetching needed (advanced)

In [12]:
sessions = acquisition.Session & (acquisition.Session.Experimenter & 'experimenter = "richarga"')

In [13]:
project_sessions = sessions.proj(project_name=f"'{project_name}'")

In [14]:
project_sessions

animal_id,datasource_id,session_time start time of this session,project_name calculated attribute
40bfa03a7a7018cc,0,2018-07-03 20:32:28,proj1


In [15]:
acquisition.ProjectSession.insert(project_sessions)

In [16]:
acquisition.ProjectSession()

project_name,animal_id,datasource_id,session_time start time of this session
proj1,40bfa03a7a7018cc,0,2018-07-03 20:32:28
proj1,724240a0a756020f,0,2018-04-24 10:25:23
